In [2]:
import pandas as pd

In [3]:
oil_consumption_url = 'http://en.wikipedia.org/wiki/List_of_countries_by_oil_consumption'
oil_production_url = 'http://en.wikipedia.org/wiki/List_of_countries_by_oil_production'
oil_reserves_url = 'http://en.wikipedia.org/wiki/List_of_countries_by_proven_oil_reserves'

In [7]:
oil_consumption_df = pd.read_html(oil_consumption_url)[0]
oil_consumption_df.rename(columns={'Oil consumption(bbl/day)':'Barrel consumption/day'},inplace=True)
oil_consumption_df['Rank'].replace('-','NaN',inplace=True)
oil_consumption_df['Barrel consumption/day'] = oil_consumption_df['Barrel consumption/day'].apply(lambda x: x.split('[')[0])
oil_consumption_df['Year'] = oil_consumption_df['Year'].apply(lambda x: x.split(' ')[0])

def remove_tags(x):
    split_string = x.split(']')
    if len(split_string[0]) != 4:
        return split_string[1]
    else:
        return split_string[0]

oil_consumption_df['Year'] = oil_consumption_df['Year'].apply(lambda x: remove_tags(x))
oil_consumption_df['Country/Region'] = oil_consumption_df['Country/Region'].apply(lambda x: x.split('(')[0])
oil_consumption_df.set_index('Country/Region',drop=True,inplace=True)
oil_consumption_df.to_csv('../data/oil_consumption_scrape_data.csv')
oil_consumption_df

,Rank,Barrel consumption/day,Year
Country/Region,,,
World,NaN,"100,100,000",2019
United States,1,20000000,2018
European Union,NaN,"15,000,000",2017
China,2,13500000,2018
India,3,4990000,2018
...,...,...,...
Dominica,205,918,2011
Sao Tome and Principe,206,904,2011
Falkland Islands,207,300,2011


In [6]:
oil_production_df = pd.read_html(oil_production_url)[0]
oil_production_df.drop(oil_production_df.columns[oil_production_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
oil_production_df.rename(columns={'Oil production2019 (bbl/day)[1]':'2019 Crude Oil Barrel production/day',
                                 'Oil production per capita2017 (bbl/day/million people)[5]':
                                  '2017 Crude Oil Barrel production/day/million people'},inplace=True)
oil_production_df['Country'] = oil_production_df['Country'].apply(lambda x: x.split('[')[0])
oil_production_df['Country'] = oil_production_df['Country'].apply(lambda x: x.split('(')[0])
oil_production_df['Country'].replace('World production','World',inplace=True)
oil_production_df.set_index('Country',drop=True,inplace=True)
oil_production_df.to_csv('../data/oil_production_scrape_data.csv')
oil_production_df

,2019 Crude Oil Barrel production/day,2017 Crude Oil Barrel production/day/million people
Country,,
World,80622000,10798
United States,15043000,35922
Saudi Arabia,12000000,324866
Russia,10800000,73292
Iraq,4451516,119664
...,...,...
Israel,390,47
Slovakia,200,37
Taiwan,196,8


In [17]:
oil_reserves_df = pd.read_html(oil_reserves_url)[0]

countries_list = oil_reserves_df['Proven reserves (millions of barrels)']['Country']
countries_list = countries_list.apply(lambda x: x.split('(')[0])
countries_list

start_2017_reserves_df = oil_reserves_df['U.S. EIA (start of 2017)[2]']
start_2017_reserves_df['Country'] = countries_list
start_2017_reserves_df['Country'].replace('World total','World',inplace=True)
start_2017_reserves_df.set_index('Country',inplace=True)

end_2017_reserves_df = oil_reserves_df['OPEC (end of 2017)[3]']
end_2017_reserves_df['Country'] = countries_list
end_2017_reserves_df['Country'].replace('World total','World',inplace=True)
end_2017_reserves_df.set_index('Country',inplace=True)

end_2015_reserves_df = oil_reserves_df['BP (end of 2015)[4]']
end_2015_reserves_df['Country'] = countries_list
end_2015_reserves_df['Country'].replace('World total','World',inplace=True)
end_2015_reserves_df.set_index('Country',inplace=True)

compiled_oil_reserves_data = pd.concat([start_2017_reserves_df,end_2017_reserves_df,end_2015_reserves_df],
                                       keys=['start_2017','end_2017','end_2015'])
compiled_oil_reserves_data.index.rename(['Year','Country'],inplace=True)
compiled_oil_reserves_data.to_csv('../data/compiled_oil_reserves_scraped_data.csv')

compiled_oil_reserves_data = pd.read_csv('../data/compiled_oil_reserves_scraped_data.csv')
compiled_oil_reserves_data = compiled_oil_reserves_data[['Country',  'Rank', 'Reserves', 'Year']]
compiled_oil_reserves_data.set_index('Country',drop=True,inplace=True)
compiled_oil_reserves_data.to_csv('../data/compiled_oil_reserves_scraped_data.csv')
compiled_oil_reserves_data

C:\Users\tmahn\Anaconda3.7\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\tmahn\Anaconda3.7\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\tmahn\Anaconda3.7\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

,Rank,Reserves,Year
Country,,,
Venezuela,1,300878.0,start_2017
Saudi Arabia,2,297700.0,start_2017
Iran,3,211600.0,start_2017
Canada,4,169709.0,start_2017
Iraq,5,142503.0,start_2017
...,...,...,...
Jordan,NaN,NaN,end_2015
Morocco,NaN,NaN,end_2015
Ethiopia,NaN,NaN,end_2015
